# Fine Tuning Open AI Models

Ang notebook na ito ay batay sa kasalukuyang gabay na ibinigay sa [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentasyon mula sa Open AI.

Pinapabuti ng fine-tuning ang pagganap ng mga foundation model para sa iyong aplikasyon sa pamamagitan ng muling pagsasanay nito gamit ang karagdagang data at konteksto na may kaugnayan sa partikular na paggamit o senaryo. Tandaan na ang mga teknik sa prompt engineering tulad ng _few shot learning_ at _retrieval augmented generation_ ay nagpapahintulot sa iyo na pagandahin ang default na prompt gamit ang kaugnay na data upang mapabuti ang kalidad. Gayunpaman, ang mga pamamaraang ito ay limitado ng max token window size ng target na foundation model.

Sa fine-tuning, epektibo nating muling sinasanay ang mismong modelo gamit ang kinakailangang data (na nagpapahintulot sa atin na gumamit ng mas maraming halimbawa kaysa sa kasya sa max token window) - at nagde-deploy ng _custom_ na bersyon ng modelo na hindi na kailangan ng mga halimbawa sa oras ng inference. Hindi lamang nito pinapabuti ang bisa ng disenyo ng ating prompt (mayroon tayong mas maraming kalayaan sa paggamit ng token window para sa iba pang bagay) kundi posibleng pinapabuti rin ang ating mga gastos (sa pamamagitan ng pagbabawas ng bilang ng mga token na kailangang ipadala sa modelo sa oras ng inference).

Ang fine tuning ay may 4 na hakbang:
1. Ihanda ang training data at i-upload ito.
1. Patakbuhin ang training job upang makakuha ng fine-tuned na modelo.
1. Suriin ang fine-tuned na modelo at ulitin para sa kalidad.
1. I-deploy ang fine-tuned na modelo para sa inference kapag nasiyahan na.

Tandaan na hindi lahat ng foundation model ay sumusuporta sa fine-tuning - [tingnan ang OpenAI dokumentasyon](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) para sa pinakabagong impormasyon. Maaari ka ring mag-fine-tune ng isang dating fine-tuned na modelo. Sa tutorial na ito, gagamitin natin ang `gpt-35-turbo` bilang target na foundation model para sa fine-tuning.

---


### Hakbang 1.1: Ihanda ang Iyong Dataset

Gumawa tayo ng chatbot na tutulong sa iyo na maunawaan ang periodic table ng mga elemento sa pamamagitan ng pagsagot sa mga tanong tungkol sa isang elemento gamit ang isang limerick. Sa _tutorial_ na ito, gagawa lang tayo ng dataset para sanayin ang modelo gamit ang ilang mga halimbawa ng mga tugon na nagpapakita ng inaasahang format ng data. Sa totoong paggamit, kakailanganin mong gumawa ng dataset na may mas maraming mga halimbawa. Maaari ka ring gumamit ng open dataset (para sa iyong domain ng aplikasyon) kung mayroon, at i-reformat ito para magamit sa fine-tuning.

Dahil nakatuon tayo sa `gpt-35-turbo` at naghahanap ng isang single-turn na tugon (chat completion), maaari tayong gumawa ng mga halimbawa gamit ang [inirerekomendang format na ito](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) na sumusunod sa mga kinakailangan ng OpenAI chat completion. Kung inaasahan mo ang multi-turn na pag-uusap, gagamitin mo ang [multi-turn example format](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) na may kasamang `weight` parameter upang ipahiwatig kung aling mga mensahe ang dapat gamitin (o hindi) sa proseso ng fine-tuning.

Gagamitin natin ang mas simpleng single-turn format para sa tutorial na ito. Ang data ay nasa [jsonl format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) na may 1 tala bawat linya, bawat isa ay kinakatawan bilang isang JSON-formatted na object. Ipinapakita ng snippet sa ibaba ang 2 tala bilang halimbawa - tingnan ang [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) para sa buong set ng halimbawa (10 halimbawa) na gagamitin natin para sa tutorial ng fine-tuning. **Tandaan:** Bawat tala _dapat_ ay nakasaad sa isang linya lamang (hindi hinati sa maraming linya tulad ng karaniwan sa isang formatted JSON file)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Sa totoong paggamit, kakailanganin mo ng mas malaking set ng mga halimbawa para sa magagandang resulta - ang palitan ay nasa pagitan ng kalidad ng mga tugon at ng oras/gastos para sa fine-tuning. Gumagamit tayo ng maliit na set upang mabilis nating matapos ang fine-tuning at maipakita ang proseso. Tingnan ang [halimbawa mula sa OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) para sa mas kumplikadong tutorial ng fine-tuning.


---

### Hakbang 1.2 I-upload ang Iyong Dataset

I-upload ang data gamit ang Files API [ayon sa paglalarawan dito](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Tandaan na upang mapatakbo ang code na ito, dapat mo munang nagawa ang mga sumusunod na hakbang:
 - Nai-install ang `openai` Python package (siguraduhing gumagamit ka ng bersyon >=0.28.0 para sa mga pinakabagong tampok)
 - Na-set ang `OPENAI_API_KEY` environment variable sa iyong OpenAI API key
Para matuto pa, tingnan ang [Setup guide](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) na ibinigay para sa kurso.

Ngayon, patakbuhin ang code upang gumawa ng file para i-upload mula sa iyong lokal na JSONL file.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Hakbang 2.1: Lumikha ng Fine-tuning na trabaho gamit ang SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Hakbang 2.2: Suriin ang Katayuan ng trabaho

Narito ang ilang bagay na maaari mong gawin gamit ang `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` - Ilista ang huling n fine-tuning na mga trabaho
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Kunin ang mga detalye ng isang partikular na fine-tuning na trabaho
- `client.fine_tuning.jobs.cancel(<job_id>)` - Kanselahin ang isang fine-tuning na trabaho
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Ilista hanggang n mga kaganapan mula sa trabaho
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Ang unang hakbang ng proseso ay _pag-validate ng training file_ upang matiyak na ang data ay nasa tamang format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Hakbang 2.3: Subaybayan ang mga kaganapan upang bantayan ang progreso


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Hakbang 2.4: Tingnan ang status sa OpenAI Dashboard


Maaari mo ring tingnan ang status sa pamamagitan ng pagbisita sa website ng OpenAI at pag-explore sa seksyon ng _Fine-tuning_ ng platform. Ipapakita nito sa iyo ang status ng kasalukuyang trabaho, at papayagan ka ring subaybayan ang kasaysayan ng mga naunang pagpapatakbo ng trabaho. Sa screenshot na ito, makikita mo na nabigo ang naunang pagpapatakbo, at nagtagumpay ang pangalawang pagpapatakbo. Bilang konteksto, nangyari ito nang ginamit ng unang pagpapatakbo ang isang JSON file na may maling format ng mga tala - nang maitama ito, matagumpay na natapos ang pangalawang pagpapatakbo at naging available ang modelo para magamit.

![Fine-tuning job status](../../../../../translated_images/tl/fine-tuned-model-status.563271727bf7bfba.webp)


Maaari mo ring tingnan ang mga mensahe ng status at mga sukatan sa pamamagitan ng pag-scroll pababa pa sa visual dashboard tulad ng ipinapakita:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/tl/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/tl/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Hakbang 3.1: Kunin ang ID at Subukan ang Fine-Tuned na Modelo sa Code


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Hakbang 3.2: I-load at Subukan ang Fine-Tuned na Modelo sa Playground

Maaari mo nang subukan ang fine-tuned na modelo sa dalawang paraan. Una, maaari kang bumisita sa Playground at gamitin ang Models drop-down upang piliin ang iyong bagong fine-tuned na modelo mula sa mga nakalistang opsyon. Ang isa pang opsyon ay gamitin ang "Playground" na opsyon na ipinapakita sa Fine-tuning panel (tingnan ang screenshot sa itaas) na naglulunsad ng sumusunod na _comparative_ na view na nagpapakita ng foundation at fine-tuned na mga bersyon ng modelo nang magkatabi para sa mabilisang pagsusuri.

![Fine-tuning job status](../../../../../translated_images/tl/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Punan lamang ang system context na ginamit sa iyong training data at ibigay ang iyong test na tanong. Mapapansin mo na parehong na-update ang magkabilang panig ng magkaparehong context at tanong. Patakbuhin ang paghahambing at makikita mo ang pagkakaiba sa mga output sa pagitan nila. _Pansinin kung paano inilalahad ng fine-tuned na modelo ang tugon sa format na ibinigay mo sa iyong mga halimbawa habang ang foundation na modelo ay sumusunod lamang sa system prompt_.

![Fine-tuning job status](../../../../../translated_images/tl/fine-tuned-playground-launch.5a26495c983c6350.webp)

Mapapansin mo rin na ang paghahambing ay nagbibigay din ng bilang ng mga token para sa bawat modelo, at ang oras na ginugol para sa inference. **Ang partikular na halimbawang ito ay isang payak na halimbawa na nilalayong ipakita ang proseso ngunit hindi talaga sumasalamin sa totoong dataset o senaryo**. Maaaring mapansin mo na parehong nagpapakita ang dalawang sample ng parehong bilang ng mga token (magkapareho ang system context at user prompt) kung saan ang fine-tuned na modelo ay mas matagal sa inference (custom na modelo).

Sa mga totoong senaryo, hindi ka gagamit ng ganitong simpleng halimbawa, kundi mag-fi-fine-tune laban sa totoong data (hal., katalogo ng produkto para sa customer service) kung saan mas magiging halata ang kalidad ng tugon. Sa _ganitong_ konteksto, ang pagkuha ng katumbas na kalidad ng tugon gamit ang foundation na modelo ay mangangailangan ng mas maraming custom prompt engineering na magpapataas ng paggamit ng token at posibleng ang kaugnay na oras ng pagproseso para sa inference. _Para subukan ito, tingnan ang mga halimbawa ng fine-tuning sa OpenAI Cookbook upang makapagsimula._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Paunawa**:
Ang dokumentong ito ay isinalin gamit ang serbisyong AI na pagsasalin na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagamat nagsusumikap kami para sa katumpakan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o di-tumpak na impormasyon. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na pangunahing sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
